Syntax natürlicher Sprachen, WS 2023/24

# 06 - Übung

In [6]:
import nltk
from nltk import Tree
from nltk import DependencyGraph
from spacy import displacy
import spacy
from itertools import chain

In [7]:
def _tree_labeled(self, i):
    node = self.get_by_address(i)
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word + '(' + rel + ')', [self._tree_labeled(dep) for dep in deps]) if deps else word + '(' + rel + ')'

def tree_labeled(self):
    node = self.root
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled

def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split(); line_list.pop(0); line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))
    return "\n".join([i for i in sent_list[0:]])

def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())
    deps = [x for x in deps if x]
    ex, word_list, arc_list = [], [], []
    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1; head = int(dep[2]); label = dep[3]
        if head>line:
            start = index; end = head-1; direction = "left"
        else:
            start = head-1; end = index; direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})
    ex.append({"words": word_list,"arcs": arc_list})
    return ex

def show_dep_trees(sent_nr,style=1):
    sent = transform_nr_conll(sent_nr)
    dg = DependencyGraph(sent)
    if style == 0 or style == 2:
        tree_labeled = dg.tree_labeled()
        tree_labeled.pretty_print(unicodelines=True)   
    if style == 1 or style == 2:
        ex = displacy_dep_input(sent)
        html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

## Aufgabe 1 - Dependenzanalyse

#### Im folgenden soll das __MALT/CONLL-Format__ für die Analyse und das Labeling von Dependenzrelationen mit folgenden Eingeschaften verwendet werden:

- pro Zeile: `Position_Wort, Wort, Position_Kopf, Label_Dependenzrelation`
- Wurzelknoten: `Position_Kopf` = 0, `Label_Dependenzrelation` = ROOT


### a) Analysieren Sie damit die Dependenzbeziehungen des folgenden Satzes im UD-Schema:
*ein elektrisches Auto fährt schnell*


In [14]:
sent_nr = """
1 ein 3 det
2 elektrisches 3 amod
3 Auto 4 nsubj
4 fährt 0 ROOT
5 schnell 4 advmod
"""

show_dep_trees(sent_nr)

### b) Analysieren Sie die Dependenzbeziehungen des folgenden Satzes im UD-Schema:
*Die neue Kollegin fährt mit der S-Bahn zu ihrem neuen Arbeitsplatz*


In [25]:
sent_nr = """
1 Die 3 det
2 neue 3 amod
3 Kollegin 4 nsubj
4 fährt 0 ROOT
5 mit 7 case
6 der 7 det
7 S-Bahn 4 obl
8 zu 11 case
9 ihrem 11 det
10 neuen 11 amod
11 Arbeitsplatz 4 obl
"""

show_dep_trees(sent_nr)

## Aufgabe 2 - Dependenzanalyse und Erweiterung

### a) Analysieren Sie die Dependenzbeziehungen dieses einfachen transitiven Satzes im UD-Schema:
*der Hund jagt den langsamen Briefträger*


In [27]:
sent_nr = """
1 der 2 det
2 Hund 3 nsubj
3 jagt 0 ROOT
4 den 6 det
5 langsamen 6 amod
6 Briefträger 3 obj
"""

show_dep_trees(sent_nr)

### b) Fügen Sie ein präpositionales Adverbial hinzu:


In [29]:
sent_nr = """
1 der 2 det
2 Hund 3 nsubj
3 jagt 0 ROOT
4 den 6 det
5 langsamen 6 amod
6 Briefträger 3 obj
7 auf 9 case
8 einer 9 det
9 Wiese 3 obl
"""

show_dep_trees(sent_nr)

### c) Fügen Sie dem Ausgangssatz ein präpositionales Attribut hinzu:


In [30]:
sent_nr = """
1 der 2 det
2 Hund 3 nsubj
3 jagt 0 ROOT
4 den 6 det
5 langsamen 6 amod
6 Briefträger 3 obj
7 aus 8 case
8 München 6 nmod
"""

show_dep_trees(sent_nr)

## Aufgabe 3 - Satzglied-Funktion


### a) Objekt vs Adverbial

#### Gegeben sei folgender Satz, dessen drittes Satzglied den *geschehens*-Test besteht (der Satz bleibt wohlgeformt). 


In [31]:
sentence = ["er", "isst", "den ganzen Tag"]

sentence[0] + " " + sentence[1] + ", und das geschieht " + sentence[2]

'er isst, und das geschieht den ganzen Tag'

#### Für welchen Typ von Dependent spricht dies (Angabe/Adjunkt oder Ergänzung/Komplement)?


In [ ]:
# Adjunkt


#### Welche syntaktische Funktion hat das dritte Satzglied hier?  (Objekt oder Adverbial?) Testen Sie über Passivierung!

In [ ]:
# Adverbial

### b) Analysieren Sie die Dependenzbeziehungen des vorherigen Satzes im UD-Schema.


In [32]:
sent_nr = """
1 er 2 nsubj
2 isst 0 ROOT
3 den 5 det
4 ganzen 5 amod
5 Tag 2 obl
"""

show_dep_trees(sent_nr)

### c) Präpositionalobjekt vs Adverbial

#### Gegeben sei folgender Satz, dessen drittes Satzglied den *geschehens*-Test nicht besteht. 


In [33]:
sentence = ["er", "wartet", "auf den Sonnenuntergang"]

sentence[0] + " " + sentence[1] + ", und das geschieht " + sentence[2]

'er wartet, und das geschieht auf den Sonnenuntergang'

#### Für welchen Typ von Dependent spricht dies (Angabe/Adjunkt oder Ergänzung/Komplement)?


In [ ]:
# Komplement


#### Welche syntaktische Funktion hat das dritte Satzglied hier?  (Objekt oder Adverbial?) Testen Sie über Erfragbarkeit mit Pronominaladverb!

In [ ]:
# Objekt (worauf wartet er?)

### d) Analysieren Sie die Dependenzbeziehungen des vorherigen Satzes im UD-Schema.

#### Beachten Sie ggf. die Analysekonvention im UD-Schema für Präpositionalobjekte.


In [35]:
sent_nr = """
1 er 2 nsubj
2 wartet 0 ROOT
3 auf 5 case
4 den 5 det
5 Sonnenuntergang 2 obl
"""

show_dep_trees(sent_nr)

### e) Adverbiales Komplement vs Adjunkt

#### Gegeben sei folgender Satz, dessen viertes Satzglied (*ins Wasser*) den *geschehens*-Test nicht besteht. 


In [36]:
sentence = ["er", "stellt", "die Blumen", "ins Wasser"]

sentence[0] + " " + sentence[1] + " " + sentence[2] + ", und das geschieht " + sentence[3]

'er stellt die Blumen, und das geschieht ins Wasser'

#### Für welchen Typ von Dependent spricht dies (Angabe/Adjunkt oder Ergänzung/Komplement)?


In [ ]:
# Komplement


#### Welche syntaktische Funktion hat das vierte Satzglied (*ins Wasser*) hier?  (Objekt oder Adverbial?) Testen Sie über Passivierung und Erfragbarkeit mit Pronominaladverb bzw. Ersatz mit einer Pronominaladverb-Nebensatz-Konstruktion!

In [ ]:
# Adverbial

### f) Analysieren Sie die Dependenzbeziehungen des vorherigen Satzes im UD-Schema.


In [37]:
sent_nr = """
1 er 2 nsubj
2 stellt 0 ROOT
3 die 4 det
4 Blumen 2 obj
5 ins 6 case
6 Wasser 2 obl
"""

show_dep_trees(sent_nr)

## *Aufgabe 4 - Parsing nicht-projektiver Strukturen*


#### Gegeben sei folgender Dependenzgraph mit nicht-projektiver Struktur:

In [38]:
sent_nr = """
1 Hunde 2 nsubj
2 bellen 0 ROOT
3 die 4 nsubj
4 beißen 1 acl:relcl
"""

show_dep_trees(sent_nr)

### a) Überprüfen Sie, ob eine solche nicht-projektive Struktur mit dem bekannten Shift-Reduce-Dependency-Parser verarbeitet werden kann, indem Sie wieder den Typ der REDUCE-Übergänge (`LEFTARC`, `RIGHTARC`) sowie die Reihenfolge deren Durchführung bei Verarbeitung dieser Struktur angeben.


In [40]:
sent_nr = """
1 Hunde 2 LEFTARC-1
2 bellen 0 ROOT
3 die 4 LEFTARC-2
4 beißen 1 -
"""

show_dep_trees(sent_nr)

## geht nicht. Nach LEFTARC-2 kann er Bezug Hunde->beißen nicht mehr gefunden werden (Stack: bellen beißen )

### b)  Schreiben Sie ein (ungelabelte) Dependenzgrammatik für die Satzstruktur:

In [42]:
grammar = nltk.DependencyGrammar.fromstring("""
    'bellen' -> 'Hunde'
    'Hunde' -> 'beißen'
    'beißen' -> 'die'
""")

### c) Testen Sie, ob der NLTK.ProjectiveDependencyParser mit Ihrer Grammatik die projektive Version des Satzes erkennt: 

In [43]:
sent = 'Hunde die beißen bellen'
print(sent)

Hunde die beißen bellen


In [44]:
parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent.split()):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

(bellen (Hunde (beißen die))) 

bellen
  │    
Hunde 
  │    
beißen
  │    
 die  



### c) Probieren Sie, die nicht-projektive Version des Satzes mit dem NLTK.ProjectiveDependencyParser sowie dem NLTK.NonprojectiveDependencyParser zu parsen und vergleichen Sie die Ergebnisse: 

In [45]:
sent = 'Hunde bellen die beißen'
print(sent)

Hunde bellen die beißen


In [46]:
parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent.split()):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

In [47]:
parser = nltk.NonprojectiveDependencyParser(grammar)
g, = parser.parse(sent.split())

print(g.root['word'])

for _, node in sorted(g.nodes.items()):
    if node['word'] is not None:
        print('{address} {word}: {d}'.format(d=node['deps'][''], **node))

print('\n', g.tree(), '\n')
g.tree().pretty_print(unicodelines=True)

bellen
1 Hunde: [4]
2 bellen: [1]
3 die: []
4 beißen: [3]

 (bellen (Hunde (beißen die))) 

bellen
  │    
Hunde 
  │    
beißen
  │    
 die  



In [65]:
# Ergebnis Vergleich: 

## *Aufgabe 5 - Valenz und Dependenz*

#### Zeigen Sie an folgendem Satz den Unterschied zwischen Valenz und Dependenz auf:
- *Peter schläft gerne im Zelt.*

In [15]:
Tree.fromstring("""(schläft Peter gerne (Zelt im))""").pretty_print(unicodelines=True)

      schläft     
  ┌──────┼─────┐   
  │      │    Zelt
  │      │     │   
Peter  gerne   im 



In [ ]:
# 

## *Aufgabe 6 - Phrasenkategorien*

#### Erläutern Sie an folgendem Beispielsatz den Unterschied zwischen Adverb und Adverbial.
- Paul schwimmt sehr gut.



In [ ]:
# Adverbial: Satzglied (sehr gut), Adverb: Wortart (sehr)

## *Aufgabe 7 - Subjektidentifikation*

#### Inwiefern bereitet die NP *ein Fehler* im folgenden Satz Schwierigkeiten für die Anwendung der gängigen Identifikationskriterien von Subjekten?
- Dem Lehrer ist schon wieder ein Fehler unterlaufen.



In [ ]:
#